In [11]:
import nbformat
import os
from crewai import Agent, Crew, Task, LLM
from crewai_tools import DirectoryReadTool
from crewai.tools import tool
from dotenv import load_dotenv

load_dotenv()
# MODEL_NAME = 'gemini/gemini-2.0-flash'
MODEL_NAME = 'gemini/gemini-2.5-flash-preview-04-17'

llm = LLM(MODEL_NAME, 
          temperature=0.5, 
          api_key=os.getenv('GEMINI_API_KEY'), 
          reasoning_effort ='low'
          )

In [12]:
@tool('Faz a leitura de um arquivo ipynb')
def ler_notebook(caminho_arquivo):
    """
    Lê um arquivo Jupyter Notebook (.ipynb) e retorna o conteúdo como um objeto Python.

    Args:
        caminho_arquivo: O caminho para o arquivo .ipynb.

    Returns:
        Um objeto NotebookNode representando o conteúdo do notebook.
    """
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            notebook = nbformat.read(file, as_version=4)  # Use version 4 (mais comum)
            return notebook
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado: {caminho_arquivo}")
        return None
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return None
    
@tool('limpeza nos notebooks ipynb')
def limpar_outputs(notebook: dict):
    """
    Limpa todos os outputs das células de código em um notebook Jupyter.
    
    Esta função percorre todas as células do notebook e remove qualquer
    saída (output) das células do tipo 'code', mantendo apenas o código-fonte.
    Útil para versionar notebooks ou reduzir o tamanho do arquivo.
    
    Args:
        notebook (nbformat.notebooknode.NotebookNode): O objeto notebook a ser processado.
        
    Returns:
        nbformat.notebooknode.NotebookNode: O notebook com os outputs limpos.
    """
    if not isinstance(notebook, nbformat.notebooknode.NotebookNode): 
        raise TypeError('Objecto de entrada não é um arquivo ipynb')
    
    for cell in notebook.cells: 
        if cell.cell_type=='code': 
            cell.outputs = []
    
    return notebook

@tool('Extrai os código fonte de todas as celulas em um notebooks Jupyter (ipynb)')
def extracao_codigo(notebook: dict): 
    """
    Extracts the source code from all code cells in a Jupyter notebook.
    
    Args:
        notebook (nbformat.notebooknode.NotebookNode): The notebook object to extract code from.
        
    Returns:
        list: A list of strings, each containing the source code from a code cell.
        
    Raises:
        TypeError: If the input is not a NotebookNode object.
    """
    
    if not isinstance(notebook, nbformat.notebooknode.NotebookNode): 
        raise TypeError('Objecto de entrada não é um arquivo ipynb')
    
    sources = []
    for cell in notebook.cells:
        if cell.cell_type=='code':
            sources.append(cell['source'])
    
    return sources

@tool('Faz as leituras dos arquivos')
def ler_arquivo(filepath: str) -> str:
    """
    Reads the content of a file and returns it as a string.
    
    Args:
        filepath: The path to the file to read.
        
    Returns:
        The content of the file as a string.
    """
    with open(filepath, 'r', encoding='utf-8') as file: 
        content = file.read()
    return content

docs_tool = DirectoryReadTool(directory='./codes')

@tool('Escreve as informações em json')
def escrever_json(filepath: str, texto: str) -> str:
    """
    Writes text content to a JSON file at the specified filepath.
    
    Args:
        filepath: The path to the file where the content should be written.
        texto: The text content to be written to the file (should be JSON-formatted).
        
    Returns:
        A confirmation message indicating that the file was written successfully.
    """
    import json
    try:
        # Parse the text as JSON to validate it
        json_data = json.loads(texto)
        
        # Write the JSON data to the file with proper formatting
        with open(filepath, 'w', encoding='utf-8') as file:
            json.dump(json_data, file, ensure_ascii=False, indent=4)
            
        return f"JSON data successfully written to {filepath}"
    except json.JSONDecodeError:
        # If the text is not valid JSON, write it as regular text
        with open(filepath, 'w', encoding='utf-8') as file:
            file.write(texto)
            
        return f"Text content written to {filepath} (not valid JSON format)"
    except Exception as e:
        return f"Error writing to file: {e}"


In [13]:
analyst = Agent(
    role="Analista de Busca", 
    goal="Encontrar qualquer arquivos no diretório `./codes`",
    backstory="Possui um mestrado em ciencia da computação, e consegue identificar qualquer arquivo de código pela sua extensão",
    tools=[docs_tool],
    llm=llm,
)

datascientist = Agent(
    role="Cientista de Dados",
    goal='Fazer a leitura dos arquivos, e identificar se ele possui algum prompt de machine learning',
    backstory='Formato em estatística, e mestre em ciencia da computação, é especialista em modelos de LLM', 
    tools = [ler_arquivo, extracao_codigo, limpar_outputs, ler_notebook], 
    llm=llm
)

construtor = Agent(
    role="Construtor",
    goal='Coletar as informações passadas pelo Cientista de Dados em salvar eles em formato json',
    backstory='Formato em ciencia da computação, e especialista em não substituir arquivos', 
    tools = [escrever_json], 
    llm=llm
)

task_analist = Task(description="Identificar quais os arquivos existem no diretório designado pelo usuário e passar para o Cientista de Dados",
                           agent=analyst, 
                           name="Directory Search", 
                           expected_output="""Caminhos dos arquivos encontrados: \n\t
                                - arquivo 1 - (caminho do arquivo)
                                - arquivo 2 - (caminho do arquivo)
                                ...
                                """)

task_ds = Task(description="Fazer a leitura dos arquivos, e identificar qual foi o prompt de LLM foi utilizado para enviar para um Modelo e passar para Construto o nome do arquivo, e qual prompt foi utilizado", 
                      agent=datascientist,
                      name='Prompt Evaluator', 
                      expected_output="""O prompt usado no arquivo `(nome do arquivo)` foi
                      \t (prompt)""")

task_contrutor = Task(description="Coletar as informações passadas pelo Cientista de dados, fazer um limpeza nas strings, e salvar o arquivo json como './prompts.json'", 
                      agent=construtor,
                      name='File Write', 
                      expected_output="Um arquivo json salvo no caminho designado", 
                      output_file="tink.txt")

crew = Crew(name="Prompt Search Crew",
                   tasks=[task_analist, task_ds, task_contrutor],
                   agents=[analyst, datascientist, construtor],
                   verbose=True
)

result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: Prompt Search Crew                                                                                       │
│  ID: bf87ba48-f8c2-4337-8035-425ae7954252                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: Prompt Search Crew
└── 📋 Task: a38eea65-7b16-477b-a67d-486720e38077
       Status: Executing Task...

🚀 Crew: Prompt Search Crew
└── 📋 Task: a38eea65-7b16-477b-a67d-486720e38077
       Status: Executing Task...
    └── 🤖 Agent: Analista de Busca
            Status: In Progress

# Agent: Analista de Busca
## Task: Identificar quais os arquivos existem no diretório designado pelo usuário e passar para o Cientista de Dados


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



🚀 Crew: Prompt Search Crew
└── 📋 Task: a38eea65-7b16-477b-a67d-486720e38077
       Status: Executing Task...
    └── 🤖 Agent: Analista de Busca
            Status: In Progress

...

 Error during LLM call: litellm.ServiceUnavailableError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}

 An unknown error occurred. Please check the details below.
 Error details: litellm.ServiceUnavailableError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}



🚀 Crew: Prompt Search Crew
└── 📋 Task: a38eea65-7b16-477b-a67d-486720e38077
       Assigned to: Analista de Busca
       Status: ❌ Failed
    └── 🤖 Agent: Analista de Busca
            Status: In Progress

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: a38eea65-7b16-477b-a67d-486720e38077                                                                     │
│  Agent: Analista de Busca                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: Prompt Search Crew                                                                                       │
│  ID: bf87ba48-f8c2-4337-8035-425ae7954252                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ServiceUnavailableError: litellm.ServiceUnavailableError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}


In [ ]:
print(result.raw)

{"prompts": [
    {"file": "./codes/placeholder_1.txt", "content": "Você é um especialista em marketing digital. Sua tarefa é gerar 3 títulos atrativos para um anúncio de e-commerce de camisetas.  \nConsidere os seguintes dados/contexto:\n\nProduto: Camiseta estampada 100% algodão\nPúblico-alvo: Jovens de 18 a 30 anos\nTom de voz: Descolado e direto\n\nCom base nisso, produza uma saída que:\n\n1. Seja criativa e persuasiva.\n2. Atenda aos seguintes critérios: máximo 60 caracteres, evitar clichês, focar no diferencial.\n3. Siga o formato de saída: lista numerada com os títulos.\n\nResponda em português e evite repetições ou floreios desnecessários."},
    {"file": "./codes/placeholder_2.ipynb", "content": "prompt_template = \"\"\nExtract the key entities (people, organizations, locations) from the following text:\n---\n{text}\n---\nReturn the result as a JSON object with keys 'people', 'organizations', 'locations'.\n\"\""},
    {"file": "./codes/placeholder_3.py", "content": "def gerar_